In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from dotenv import load_dotenv
import datetime

In [2]:
load_dotenv()
jdbc_driver_path = "D:/Data_Engineering/SQL_and_PySpark/postgresql-42.7.4.jar"

In [3]:
spark = SparkSession.builder \
    .appName('Solution') \
    .config("spark.driver.extraClassPath", jdbc_driver_path) \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .getOrCreate()

In [4]:
# Define JDBC properties
jdbcHostname = os.getenv("HOST")
jdbcDatabase = os.getenv("DB_NAME")
jdbcUsername = os.getenv("USER")
jdbcPassword = os.getenv("PASSWORD")
jdbcPort = 5432 
jdbcDriver = "org.postgresql.Driver"

connProperties = {
  "user": jdbcUsername,
  "password": jdbcPassword,
  "driver": jdbcDriver
}

jdbcUrl = f"jdbc:postgresql://{jdbcHostname}:{jdbcPort}/{jdbcDatabase}"


In [5]:
sql = "(SELECT * FROM artist) AS artist_alias"
artist_df = spark.read.jdbc(url = jdbcUrl, table = sql, properties = connProperties)

In [6]:
artist_df.show()

+---------+--------------------+----------+------------+------------+-----------+--------------------+-----+-----+
|artist_id|           full_name|first_name|middle_names|   last_name|nationality|               style|birth|death|
+---------+--------------------+----------+------------+------------+-----------+--------------------+-----+-----+
|      500|Pierre-Auguste Re...|    Pierre|     Auguste|      Renoir|     French|       Impressionist| 1841| 1919|
|      501|   Alexandre Cabanel| Alexandre|        NULL|     Cabanel|     French|          Classicist| 1823| 1889|
|      502|         James Ensor|     James|        NULL|       Ensor|    Belgian|       Expressionist| 1860| 1949|
|      503|     Maximilien Luce|Maximilien|        NULL|        Luce|     French|         Pointillist| 1858| 1941|
|      504|        August Macke|    August|        NULL|       Macke|     German|       Expressionist| 1887| 1914|
|      505| Thomas Gainsborough|    Thomas|        NULL|Gainsborough|    English